<a href="https://colab.research.google.com/github/House-Plants/Chemistry-helper/blob/main/CHEM_ANALYSE_V2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <style>
    body {background-color: powderblue;}
    h1   {color: blue;}
    p    {color: red;}
  </style>
 <body>
  <h1 style='font-family: Helvetica, Arial, sans-serif;'>Chemistry Helper!</h1>
  <p>
    This code aims to help and assist you with your endeavour in chemistry. <br>
    Coded by <b>Gan Kai</b>, with aid from stack exchange.
  </p>
 </body>
</head>


In [74]:
import math,re
from collections import Counter
from google.colab import output
import numpy as np
from scipy.special import genlaguerre, lpmv
import matplotlib.pyplot as plt
import time
import sympy as sp
from IPython.display import display, Math

class draw_struc:
  def __init__(self,CSAL):
    self.bonds={'·':[],'◦':[],'•':[]}
    self.points=[]
    self.draw_(CSAL)
    self.draw_out()

  def draw_(self,csal,ref=[0,0]):
    secs=self.disect(csal); bon_t={1:'·',2:'◦',3:'•'}
    try: self.points.append(ref+[secs.pop(0)])
    except: print('ERROR IN',csal)
    for i in secs:
      bonding, dir, fp=self.identify(i)
      dy,dx=dir
      self.bonds[bon_t.get(bonding,'·')].append([ref[0]+dy,ref[1]+dx])
      new_csal=i[fp:-1]
      self.draw_(new_csal,ref=[ref[0]+dy*2,ref[1]+dx*2])
    return

  def identify(self,csal):
    bonding=0;direction=[-1,0];final_p=0
    for v,i in enumerate(csal):
      if i=='*': direction[0]=-1
      elif i=='>': direction[1]=1
      elif i=='<': direction[1]=-1
      elif i=='.': direction[0]=1
      elif i=='-': direction[0]=0
      elif i.isdigit(): bonding=int(i)
      elif i=='[':
        final_p=v+1
        break
    return bonding, direction, final_p

  def disect(self,part):
    sections=['']; counter=0
    for i in part:
      if i in '*><.-' and not counter and sections[-1] not in '*><.-': sections.append(i)
      else:sections[-1]+=i
      if i==']': counter-=1
      elif i=='[': counter+=1
    return sections

  def draw_out(self):
    mx=min([i[1] for i in self.points])
    my=min([i[0] for i in self.points])
    Mx=max([i[1] for i in self.points])+abs(mx)+1
    My=max([i[0] for i in self.points])+abs(my)+1
    grid=[[' ' for __ in range(Mx)] for _ in range(My)]
    for i in self.points: grid[i[0]+abs(my)][i[1]+abs(mx)]=i[2]
    for i in self.bonds:
      for j in self.bonds[i]:
        grid[j[0]+abs(my)][j[1]+abs(mx)]=i
    for _ in grid:
      print(end=' ')
      for __ in _:
        print(end=__+' ')
      print()

class chem:
  def __init__(self):
    self.main_equa='(NONE)'.center(46)
    self.lobby_()

  def lobby_(self):
    while True:
      output.clear()
      print('          CHEMICAL EQUATION ANALYTIC          ')
      print('----------------------------------------------')
      print(self.main_equa)
      print('----------------------------------------------')
      print('[1]: Set chemical equation🧪')
      print('[2]: Individual analysis')
      if '(NONE)' in self.main_equa: print(end='\033[90m')
      print('[3]: Equation analysis👩🏻‍🔬\033[0m')
      print('[4]: Drawing compounds')
      print('[5]: Visualising Electron clouds')
      print('[6]: Visualising a particle in a box📦')
      print()
      print('[f]: Formula Sheet📝')
      con_in=input()
      if con_in=='1': self.set_equa()
      elif con_in=='2': self.indi_search()
      elif con_in=='3' and '(NONE)' not in self.main_equa: self.equa_tackle()
      elif con_in=='4': self.drawing_act()
      elif con_in=='5': self.quan_draw()
      elif con_in=='6': self.box_part()
      elif con_in.lower()=='f':
        output.clear()
        self.display_form_sheet()
        input('[ENTER TO RETURN]')
  def set_equa(self):
    output.clear()
    print('SETTING CHEMICAL EQUATION'.center(46))
    print('----------------------------------------------')
    print('e.g,HCl+NaOH>NaCl+H2O (reactants)>(products)')
    print('Program will auto-balance')
    equa=input()
    self.data_ori=equa
    self.local_values(equa)
    self.collectervise_diff()
    print(self.local_left,self.local_right)
    self.reorg()
  def indi_search(self):
    while True:
      output.clear()
      print('SEARCHING ELEMENT🔍'.center(46))
      print('----------------------------------------------')
      print('Enter formula of element (type exit to leave)')
      symbol = input('>').replace(' ','')
      if self.look_avail(symbol):
        output.clear()
        print('Search successful!')
        print('----------------------------------------------')
        print(self.look_name(symbol,ntype='name').ljust(15),'['+self.look_name(symbol,ntype='form')+']')
        print('----------------------------------------------')
        symbol=self.look_name(symbol,ntype='form')
        print('Category:',self.look_cate(symbol))
        print('Atomic mass:',self.look_RAM(symbol))
        print('Atomic number:',self.look_AN(symbol))
        print('Electorn filling:',self.shell_sorter(self.look_AN(symbol)))
        input('[ENTER TO RETURN]')
      elif 'exit' in symbol.lower(): return
      else:
        print('Search was unsuccessful...')
        print(symbol, 'does not exist in our database')
        input('[ENTER TO RETURN]')
  def equa_tackle(self):
    output.clear()
    print('----------------------------------------------')
    print(self.main_equa)
    print()
    print('CHEMICAL REACTION TYPE:',self.chem_re_type())
    print('Verification')
    self.verifying_nums()
    self.create_table()
    self.set_structure()
    self.set_states()
    self.set_molar_ratio()
    print('\nEquation with state symbols')
    bews=list(self.main_equa); counter=0; cap=''
    for i in bews:
      if i=='+':
        print(end=cap+' ('+self.states_cus[counter]+') ')
        print(end='+ ')
        cap=''; counter+=1
      elif i=='>':
        print(end=cap+' ('+self.states_cus[counter]+') ')
        print(end='> ')
        cap=''; counter+=1
      else:
        cap+=i
    print(cap+' ('+self.states_cus[counter]+') '+'\n')
    if not self.approval: print('\033[3m!!: Equation seems to be unbalanced\033[0m')
    elif self.approval:
      try:
        self.mass=float(input('Reacted mass(grams): '))
        self.set_indi_RAM()
        self.set_indi_mass()
        self.display_table()
      except: print('[ERROR] encountered')
    print()
    input('[ENTER TO RETURN]')
  def drawing_act(self):
    while True:
      output.clear()
      print('✍️SKETCHING✏️'.center(46))
      print('----------------------------------------------')
      print("Type 'help' for instructions")
      while True:
        csal=input('CSAL> ').replace(' ','')
        if csal.lower()=='help':
          print("""
    Functions:
     - 'exit' to leave
     - 'clear' to clear all text
     - 'help' to call upon help menu

    Drawing code:
     - '>' to the right  (by default upwards)
     - '<' to the left (by default upwards)
     - '*' to the up
     - '.' to the bottom
     - '-' no verticle change
     - '1'/'2'/'3' to indicate the number of bonds

    There are multiple ways to draw compounds.
    Say a H2O molecules is:
      O>[H].>[H]
      O.>[H]>[H]
      H.>[O>[H]]
      ...
    These are all possible ways to draw H2O
    For example, O>[H].>[H]
      O is the parent
      Next, H is drawn on the right-upwards
      Next, H is drawn on the right-downards

    Another example, NH3, can be expressed as
    N*[H].<[H].>[H]
      N is the parent
      Next, H is drawn above
      Next, H is drawn on the left-downwards
      Next, H is drawn on the right-downwards

    Another example, Butane, can be expressed
    as C.>[C>[C.>[C]]]
      C is the parent
      Next, C is drawn on the right-downwards,
      this C is now the referenced parent.
      Next, C is drawn on the right-upwards
      refering to the previous C, this C is
      now the referenced parent.
      Next, C is drawn on the right-downwards.

    Another example, N2, is N->3[N]
      N is the parent
      Next, N is drawn on the right with
      triple bonds.

    It follows this pattern
    Parent >[ Parent >[Parent ...] ] >[ Parent >[Parent ...] ] ...

    Which looks like this in a tree-like
    format:
    Parent
      >[ Parent
        >[Parent
            ...
       ] ]

      >[ Parent
        >[Parent
            ...
       ] ]

      ...""")
        elif csal.lower()=='exit': return
        elif csal.lower()=='clear': break
        else:
          print('DRAWING OF',self.csal_to_form(csal))
          print()
          draw_struc(csal)
          print()
  def quan_draw(self):
    output.clear()
    print('VISUALISATION OF ELECTRON CLOUD'.center(46))
    print('----------------------------------------------')
    print('Type Atom and shell i.e.H3p,0')
    atm=input()
    try:
        atom_d,arrange=atm.split(',')
        lV={'s':0,'p':1,'d':2,'f':3}
        atomic_n=self.look_AN(atom_d[:-2])
        n=int(atom_d[-2])
        l=lV[atom_d[-1]]
        arrange=int(arrange)
        if -l<=arrange<=l:
          output.clear()
          self.display_elec_cloud(N=n,L=l,M=arrange,Z=atomic_n)
          time.sleep(1)
        else: raise 'err'
    except: print('Input error')
    input('[ENTER TO RETURN]')
  def box_part(self):
    output.clear()
    print('📦PARTICLE IN POTENTIAL WELL📦'.center(46))
    print('----------------------------------------------')
    print('x by y by z -length & quantum number')
    print('i.e. (3,3,3)&(1,2,1)')
    box=input()
    try:
        box_di,n_box=box.split('&')
        dimen=box_di[1:-1].split(',')
        n_box=n_box[1:-1].split(',')
        if len(dimen)==3:
          Lx,Ly,Lz=[int(i) for i in dimen]
          nx,ny,nz=[int(i) for i in n_box]
          output.clear()
          self.display_particle_box(Lx=Lx,Ly=Ly,Lz=Lz,nx=nx,ny=ny,nz=nz)
          time.sleep(1)
        else: raise 'err'
    except: print('Input error')
    input('[ENTER TO RETURN]')


  def look_RAM(self,symbol):
    atomic_masses = {
          "Ac": 227.0,     "Ag": 107.87,   "Al": 26.982,   "Am": 243.0,    "Ar": 39.948,
          "As": 74.922,    "At": 210.0,    "Au": 196.97,   "B": 10.81,     "Ba": 137.33,
          "Be": 9.0122,    "Bh": 270.0,    "Bi": 208.98,   "Bk": 247.0,    "Br": 79.904,
          "C": 12.011,     "Ca": 40.078,   "Cd": 112.41,   "Ce": 140.12,   "Cf": 251.0,
          "Cl": 35.45,     "Cm": 247.0,    "Cn": 285.0,    "Co": 58.933,   "Cr": 51.996,
          "Cs": 132.91,    "Cu": 63.546,   "Db": 268.0,    "Ds": 281.0,    "Dy": 162.50,
          "Er": 167.26,    "Es": 252.0,    "Eu": 151.96,   "F": 18.998,    "Fe": 55.845,
          "Fl": 289.0,     "Fm": 257.0,    "Fr": 223.0,    "Ga": 69.723,   "Gd": 157.25,
          "Ge": 72.630,    "H": 1.008,     "He": 4.0026,   "Hf": 178.49,   "Hg": 200.59,
          "Ho": 164.93,    "Hs": 277.0,    "I": 126.90,    "In": 114.82,   "Ir": 192.22,
          "K": 39.098,     "Kr": 83.798,   "La": 138.91,   "Li": 6.94,     "Lr": 266.0,
          "Lu": 174.97,    "Lv": 293.0,    "Mc": 290.0,    "Md": 258.0,    "Mg": 24.305,
          "Mn": 54.938,    "Mo": 95.95,    "Mt": 278.0,    "N": 14.007,    "Na": 22.990,
          "Nb": 92.906,    "Nd": 144.24,   "Ne": 20.180,   "Nh": 286.0,    "Ni": 58.693,
          "No": 259.0,     "Np": 237.0,    "O": 15.999,    "Og": 294.0,    "Os": 190.23,
          "P": 30.974,     "Pa": 231.04,   "Pb": 207.2,    "Pd": 106.42,   "Pm": 145.0,
          "Po": 209.0,     "Pr": 140.91,   "Pt": 195.08,   "Pu": 244.0,    "Ra": 226.0,
          "Rb": 85.468,    "Re": 186.21,   "Rf": 267.0,    "Rg": 282.0,    "Rh": 102.91,
          "Rn": 222.0,     "Ru": 101.07,   "S": 32.06,     "Sb": 121.76,   "Sc": 44.956,
          "Se": 78.971,    "Sg": 269.0,    "Si": 28.085,   "Sm": 150.36,   "Sn": 118.71,
          "Sr": 87.62,     "Ta": 180.95,   "Tb": 158.93,   "Tc": 98.0,     "Te": 127.60,
          "Th": 232.04,    "Ti": 47.867,   "Tl": 204.38,   "Tm": 168.93,   "Ts": 294.0,
          "U": 238.03,     "V": 50.942,    "W": 183.84,    "Xe": 131.29,   "Y": 88.906,
          "Yb": 173.05,    "Zn": 65.38,    "Zr": 91.224
        }
    return atomic_masses.get(symbol, 0)
  def look_AN(self,symbol):
    atomic_number = {
          'H':  1,   'He': 2,   'Li': 3,   'Be': 4,   'B':  5,
          'C':  6,   'N':  7,   'O':  8,   'F':  9,   'Ne': 10,
          'Na': 11,  'Mg': 12,  'Al': 13,  'Si': 14,  'P':  15,
          'S':  16,  'Cl': 17,  'Ar': 18,  'K':  19,  'Ca': 20,
          'Sc': 21,  'Ti': 22,  'V':  23,  'Cr': 24,  'Mn': 25,
          'Fe': 26,  'Co': 27,  'Ni': 28,  'Cu': 29,  'Zn': 30,
          'Ga': 31,  'Ge': 32,  'As': 33,  'Se': 34,  'Br': 35,
          'Kr': 36,  'Rb': 37,  'Sr': 38,  'Y':  39,  'Zr': 40,
          'Nb': 41,  'Mo': 42,  'Tc': 43,  'Ru': 44,  'Rh': 45,
          'Pd': 46,  'Ag': 47,  'Cd': 48,  'In': 49,  'Sn': 50,
          'Sb': 51,  'Te': 52,  'I':  53,  'Xe': 54,  'Cs': 55,
          'Ba': 56,  'La': 57,  'Ce': 58,  'Pr': 59,  'Nd': 60,
          'Pm': 61,  'Sm': 62,  'Eu': 63,  'Gd': 64,  'Tb': 65,
          'Dy': 66,  'Ho': 67,  'Er': 68,  'Tm': 69,  'Yb': 70,
          'Lu': 71,  'Hf': 72,  'Ta': 73,  'W':  74,  'Re': 75,
          'Os': 76,  'Ir': 77,  'Pt': 78,  'Au': 79,  'Hg': 80,
          'Tl': 81,  'Pb': 82,  'Bi': 83,  'Po': 84,  'At': 85,
          'Rn': 86,  'Fr': 87,  'Ra': 88,  'Ac': 89,  'Th': 90,
          'Pa': 91,  'U':  92,  'Np': 93,  'Pu': 94,  'Am': 95,
          'Cm': 96,  'Bk': 97,  'Cf': 98,  'Es': 99,  'Fm': 100,
          'Md': 101, 'No': 102, 'Lr': 103, 'Rf': 104, 'Db': 105,
          'Sg': 106, 'Bh': 107, 'Hs': 108, 'Mt': 109, 'Ds': 110,
          'Rg': 111, 'Cn': 112, 'Fl': 114, 'Lv': 116, 'Ts': 117,
          'Og': 118
        }
    return atomic_number.get(symbol, 0)
  def look_cate(self,symbol):
    cate={
        'alkali metals':[
              'Li','Na','K','Rb','Cs','Fr'
        ],
        'alkaline earth metals':[
              'Be','Mg','Ca','Sr','Ba','Ra'
        ],
        'transition metals':[
              'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
              'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd',
              'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
              'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn'
        ],
        'post-transition metals':[
              'Al', 'Ga', 'In', 'Sn', 'Tl', 'Pb', 'Bi', 'Nh', 'Fl', 'Mc',
              'Lv'
        ],
        'metalloids':[
              'B', 'Si', 'Ge', 'As', 'Sb', 'Te'
        ],
        'reactive non-metals':[
              'H', 'C', 'N', 'O', 'F', 'P', 'S', 'Cl', 'Se', 'Br', 'I',
              'Ts'
        ],
        'noble gases':[
              'He', 'Ne', 'Ar', 'Kr', 'Xe', 'Rn', 'Og'
        ],
        'lanthanides':[
              'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb',
              'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu'
        ],
        'actinides':[
              'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk',
              'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr'
        ]
    }
    for cat in cate:
      if symbol in cate[cat]: return cat
    return False
  def look_name(self,symbol,ntype='form'):
    element_names = {
    'H': 'Hydrogen', 'He': 'Helium', 'Li': 'Lithium', 'Be': 'Beryllium',
    'B': 'Boron', 'C': 'Carbon', 'N': 'Nitrogen', 'O': 'Oxygen',
    'F': 'Fluorine', 'Ne': 'Neon', 'Na': 'Sodium', 'Mg': 'Magnesium',
    'Al': 'Aluminium', 'Si': 'Silicon', 'P': 'Phosphorus', 'S': 'Sulfur',
    'Cl': 'Chlorine', 'Ar': 'Argon', 'K': 'Potassium', 'Ca': 'Calcium',
    'Sc': 'Scandium', 'Ti': 'Titanium', 'V': 'Vanadium', 'Cr': 'Chromium',
    'Mn': 'Manganese', 'Fe': 'Iron', 'Co': 'Cobalt', 'Ni': 'Nickel',
    'Cu': 'Copper', 'Zn': 'Zinc', 'Ga': 'Gallium', 'Ge': 'Germanium',
    'As': 'Arsenic', 'Se': 'Selenium', 'Br': 'Bromine', 'Kr': 'Krypton',
    'Rb': 'Rubidium', 'Sr': 'Strontium', 'Y': 'Yttrium', 'Zr': 'Zirconium',
    'Nb': 'Niobium', 'Mo': 'Molybdenum', 'Tc': 'Technetium', 'Ru': 'Ruthenium',
    'Rh': 'Rhodium', 'Pd': 'Palladium', 'Ag': 'Silver', 'Cd': 'Cadmium',
    'In': 'Indium', 'Sn': 'Tin', 'Sb': 'Antimony', 'Te': 'Tellurium',
    'I': 'Iodine', 'Xe': 'Xenon', 'Cs': 'Cesium', 'Ba': 'Barium',
    'La': 'Lanthanum', 'Ce': 'Cerium', 'Pr': 'Praseodymium', 'Nd': 'Neodymium',
    'Pm': 'Promethium', 'Sm': 'Samarium', 'Eu': 'Europium', 'Gd': 'Gadolinium',
    'Tb': 'Terbium', 'Dy': 'Dysprosium', 'Ho': 'Holmium', 'Er': 'Erbium',
    'Tm': 'Thulium', 'Yb': 'Ytterbium', 'Lu': 'Lutetium', 'Hf': 'Hafnium',
    'Ta': 'Tantalum', 'W': 'Tungsten', 'Re': 'Rhenium', 'Os': 'Osmium',
    'Ir': 'Iridium', 'Pt': 'Platinum', 'Au': 'Gold', 'Hg': 'Mercury',
    'Tl': 'Thallium', 'Pb': 'Lead', 'Bi': 'Bismuth', 'Po': 'Polonium',
    'At': 'Astatine', 'Rn': 'Radon', 'Fr': 'Francium', 'Ra': 'Radium',
    'Ac': 'Actinium', 'Th': 'Thorium', 'Pa': 'Protactinium', 'U': 'Uranium',
    'Np': 'Neptunium', 'Pu': 'Plutonium', 'Am': 'Americium', 'Cm': 'Curium',
    'Bk': 'Berkelium', 'Cf': 'Californium', 'Es': 'Einsteinium', 'Fm': 'Fermium',
    'Md': 'Mendelevium', 'No': 'Nobelium', 'Lr': 'Lawrencium', 'Rf': 'Rutherfordium',
    'Db': 'Dubnium', 'Sg': 'Seaborgium', 'Bh': 'Bohrium', 'Hs': 'Hassium',
    'Mt': 'Meitnerium', 'Ds': 'Darmstadtium', 'Rg': 'Roentgenium', 'Cn': 'Copernicium',
    'Fl': 'Flerovium', 'Lv': 'Livermorium', 'Ts': 'Tennessine', 'Og': 'Oganesson'
    }
    for i in element_names:
      if symbol.lower() in [i.lower(), element_names[i].lower()]:
        if ntype=='form': return i
        else: return element_names[i]
    return 'ERROR'
  def look_avail(self,symbol):
    element_names = {
    'H': 'Hydrogen', 'He': 'Helium', 'Li': 'Lithium', 'Be': 'Beryllium',
    'B': 'Boron', 'C': 'Carbon', 'N': 'Nitrogen', 'O': 'Oxygen',
    'F': 'Fluorine', 'Ne': 'Neon', 'Na': 'Sodium', 'Mg': 'Magnesium',
    'Al': 'Aluminium', 'Si': 'Silicon', 'P': 'Phosphorus', 'S': 'Sulfur',
    'Cl': 'Chlorine', 'Ar': 'Argon', 'K': 'Potassium', 'Ca': 'Calcium',
    'Sc': 'Scandium', 'Ti': 'Titanium', 'V': 'Vanadium', 'Cr': 'Chromium',
    'Mn': 'Manganese', 'Fe': 'Iron', 'Co': 'Cobalt', 'Ni': 'Nickel',
    'Cu': 'Copper', 'Zn': 'Zinc', 'Ga': 'Gallium', 'Ge': 'Germanium',
    'As': 'Arsenic', 'Se': 'Selenium', 'Br': 'Bromine', 'Kr': 'Krypton',
    'Rb': 'Rubidium', 'Sr': 'Strontium', 'Y': 'Yttrium', 'Zr': 'Zirconium',
    'Nb': 'Niobium', 'Mo': 'Molybdenum', 'Tc': 'Technetium', 'Ru': 'Ruthenium',
    'Rh': 'Rhodium', 'Pd': 'Palladium', 'Ag': 'Silver', 'Cd': 'Cadmium',
    'In': 'Indium', 'Sn': 'Tin', 'Sb': 'Antimony', 'Te': 'Tellurium',
    'I': 'Iodine', 'Xe': 'Xenon', 'Cs': 'Cesium', 'Ba': 'Barium',
    'La': 'Lanthanum', 'Ce': 'Cerium', 'Pr': 'Praseodymium', 'Nd': 'Neodymium',
    'Pm': 'Promethium', 'Sm': 'Samarium', 'Eu': 'Europium', 'Gd': 'Gadolinium',
    'Tb': 'Terbium', 'Dy': 'Dysprosium', 'Ho': 'Holmium', 'Er': 'Erbium',
    'Tm': 'Thulium', 'Yb': 'Ytterbium', 'Lu': 'Lutetium', 'Hf': 'Hafnium',
    'Ta': 'Tantalum', 'W': 'Tungsten', 'Re': 'Rhenium', 'Os': 'Osmium',
    'Ir': 'Iridium', 'Pt': 'Platinum', 'Au': 'Gold', 'Hg': 'Mercury',
    'Tl': 'Thallium', 'Pb': 'Lead', 'Bi': 'Bismuth', 'Po': 'Polonium',
    'At': 'Astatine', 'Rn': 'Radon', 'Fr': 'Francium', 'Ra': 'Radium',
    'Ac': 'Actinium', 'Th': 'Thorium', 'Pa': 'Protactinium', 'U': 'Uranium',
    'Np': 'Neptunium', 'Pu': 'Plutonium', 'Am': 'Americium', 'Cm': 'Curium',
    'Bk': 'Berkelium', 'Cf': 'Californium', 'Es': 'Einsteinium', 'Fm': 'Fermium',
    'Md': 'Mendelevium', 'No': 'Nobelium', 'Lr': 'Lawrencium', 'Rf': 'Rutherfordium',
    'Db': 'Dubnium', 'Sg': 'Seaborgium', 'Bh': 'Bohrium', 'Hs': 'Hassium',
    'Mt': 'Meitnerium', 'Ds': 'Darmstadtium', 'Rg': 'Roentgenium', 'Cn': 'Copernicium',
    'Fl': 'Flerovium', 'Lv': 'Livermorium', 'Ts': 'Tennessine', 'Og': 'Oganesson'
    }
    avail=[i.lower() for i in element_names]+[element_names[i].lower() for i in element_names]
    if symbol.lower() in  avail: return True
    return False
  def shell_sorter(self,AN):
    shells={
        '1s':0,'2s':0,'2p':0,'3s':0,'3p':0,'4s':0,'3d':0,
        '4p':0,'5s':0,'4d':0,'5p':0,'6s':0,'4f':0,'5d':0,
        '6p':0,'7s':0,'5f':0,'6d':0,'7p':0
    }
    cap={'s':2,'p':6,'d':10,'f':14}
    ss={'s':'a','p':'b','d':'c','f':'d'}
    for i in shells:
      if AN<=0: break
      shells[i]=min(AN,cap[i[1]])
      AN-=cap[i[1]]
    return ' '.join([i+self.super_num(str(shells[i])) if shells[i] else '' for i in sorted(shells, key=lambda x: x[0]+ss[x[1]])])
  def super_num(self,text):
    superscr=str.maketrans('1234567890', '¹²³⁴⁵⁶⁷⁸⁹⁰')
    return text.translate(superscr)

  def chem_re_type(self):
    copy_main=self.main_equa[::1]
    rea, pro =copy_main.split(">")
    rea = rea.split("+")
    for v,i in enumerate(rea):
      if i[0].isdigit(): rea[v]=i[1:]
    pro = pro.split("+")
    for v,i in enumerate(pro):
      if i[0].isdigit(): pro[v]=i[1:]

    if len(rea)>1 and len(pro)==1:
      if all([len(list(i))==1 for i in self.left_basis]): return 'Redox'
      return 'Synthesis'
    elif len(rea)==1 and len(pro)>1: return 'Decomposition'
    if 'O2' in rea and 'H2O' in pro:
      if 'CO' in pro or 'C' in pro: return 'Incomplete combustion'
      elif 'CO2' in pro: return 'Complete combustion'
      else: return 'Unknown combustion'
    if 'H2O' in pro: return 'Neutralization'
    if len(rea)==len(pro):
      if any([len(list(i))==1 for i in self.left_basis]): return 'Single Displacement'
      elif all([len(list(i))>1 for i in self.left_basis]) and all([len(list(i))>1 for i in self.right_basis]): return 'Double Displacement'
      else: return 'Displacement-type'
    return 'Unknown'
  def verifying_nums(self):
    C_left=dict(sum((Counter(i) for i in self.local_left), Counter()))
    C_right=dict(sum((Counter(i) for i in self.local_right), Counter()))
    self.approval=True
    for v,i in enumerate(C_left):
      print(f'{v+1}. {i}:',C_left[i],'(reactants)',end=' ')
      if C_left[i]==C_right[i]: print(end='= ')
      else:
        self.approval=False
        print('! ')
      print(C_right[i],'(products)')

  def create_table(self):
    copy_main=self.main_equa[::1]
    rea, pro =copy_main.split(">")
    rea = rea.split("+")
    for v,i in enumerate(rea):
      if i[0].isdigit(): rea[v]=i[1:]
    pro = pro.split("+")
    for v,i in enumerate(pro):
      if i[0].isdigit(): pro[v]=i[1:]
    self.table={}
    for i in rea: self.table[i]=[]
    self.table['empty']=[]
    for i in pro: self.table[i]=[]
  def set_molar_ratio(self):
    sub_table=[]; self.ratio=[]
    for v,i in enumerate(self.local_left):
      for name in i:
        mu=self.local_left[v][name]//self.left_basis[v][name]
        sub_table.append(f'{mu} mol')
        self.ratio.append(mu)
        break
    sub_table.append('empty')
    for v,i in enumerate(self.local_right):
      for name in i:
        mu=self.local_right[v][name]//self.right_basis[v][name]
        sub_table.append(f'{mu} mol')
        self.ratio.append(mu)
        break
    for i in self.table:
      self.table[i].append(sub_table.pop(0))
      self.table[i].append('empty')
  def set_structure(self):
    copy_main=self.main_equa[::1]
    rea, pro =copy_main.split(">")
    rea = rea.split("+")
    for v,i in enumerate(rea):
      if i[0].isdigit(): rea[v]=i[1:]
    pro = pro.split("+")
    for v,i in enumerate(pro):
      if i[0].isdigit(): pro[v]=i[1:]
    sub_table=[]
    for i in rea:
      sub_table.append(self.structure_identity(i))
    sub_table.append('empty')
    for i in pro:
      sub_table.append(self.structure_identity(i))
    for i in self.table:
      self.table[i].append(sub_table.pop(0))
  def set_indi_RAM(self):
    total_set=self.left_basis+self.right_basis;v_emp=0
    v_emp=0
    for v,name in enumerate(self.table):
      if name!='empty':
        MR=0
        for j in total_set[v-v_emp]: MR+=total_set[v-v_emp][j]*self.look_RAM(j)
        self.table[name].append(str(self.round_sf(MR))+'g/mol')
        self.table[name].append('empty')
      else:
        v_emp=1
        self.table[name].append('empty')
        self.table[name].append('empty')
  def set_indi_mass(self):
    total_set=self.left_basis+self.right_basis;v_emp=0
    for v,main_name in enumerate(self.table):
      if main_name!='empty':
        MR=0;k_emp=0
        for j in total_set[v-v_emp]: MR+=total_set[v-v_emp][j]*self.look_RAM(j)
        mols=self.mass/MR
        main_R=self.ratio[v-v_emp]
        for k,sub_name in enumerate(self.table):
          if sub_name!='empty':
            AR=0
            for m in total_set[k-k_emp]: AR+=total_set[k-k_emp][m]*self.look_RAM(m)
            Mr=(self.ratio[k-k_emp]/main_R)*mols
            self.table[sub_name].append(str(self.round_sf(Mr*AR))+'g')
            self.table[sub_name].append(str(self.round_sf(Mr))+'mol')
            self.table[sub_name].append('empty')
          else:
            k_emp=1
            for repeat in range(3):
              self.table[sub_name].append('empty')
      else:
        v_emp=1
  def set_states(self):
    copy_main=self.main_equa[::1]
    rea, pro =copy_main.split(">")
    rea = rea.split("+")
    trans={
        's':'Solid','l':'Liquid','g':'Gaseous','aq':'Aqeuous'
    }
    self.states_cus=[]
    for v,i in enumerate(rea):
      if i[0].isdigit(): rea[v]=i[1:]
    pro = pro.split("+")
    for v,i in enumerate(pro):
      if i[0].isdigit(): pro[v]=i[1:]
    sub_table=[]
    for i in rea:
      sub_table.append(trans.get(self.guess_state(i),'NID unknown'))
      self.states_cus.append(self.guess_state(i))
    sub_table.append('empty')
    for i in pro:
      sub_table.append(trans.get(self.guess_state(i),'NID unknown'))
      self.states_cus.append(self.guess_state(i))
    for i in self.table:
      self.table[i].append(sub_table.pop(0))

  def guess_state(self,form):
    chem_react=self.chem_re_type()

    metal_elements = [
          "Li", "Na", "K", "Rb", "Cs", "Fr",
          "Be", "Mg", "Ca", "Sr", "Ba", "Ra",
          "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn",
          "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd",
          "La", "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg",
          "Ac", "Rf", "Db", "Sg", "Bh", "Hs", "Mt", "Ds", "Rg", "Cn",
          "Al", "Ga", "In", "Sn", "Tl", "Pb", "Bi", "Po", "Fl",
          "Ce", "Pr", "Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu",
          "Th", "Pa", "U", "Np", "Pu", "Am", "Cm", "Bk", "Cf", "Es", "Fm", "Md", "No", "Lr"
        ]

    super_sol_cat=[
        'Li','Na','K','Rb','Cs','Fr','NH4','NO3'
    ]
    confirmed={
          "H2": "g",   "N2": "g",    "O2": "g",    "F2": "g",        "Cl2": "g",
          "CO2": "g",  "CO": "g",    "CH4": "g",
          "C2H6": "g", "C3H8": "g",  "C4H10": "g", "H2O": "l",       "HF": "l",
          "CH3OH": "l","C2H5OH": "l","C6H6": "l",  "C5H12": "l",     "C6H12O6": "s",
          "I2": "s",   "P4": "s",    "S8": "s",    "C12H22O11": "s", "C10H16N5O13P3": "s",
      }
    acids = [
          "HCl",   "HBr",   "HI", "HNO3",   "H2SO4", "HClO4", "CH3COOH",
          "H2CO3", "H3PO4", "HF", "H2C2O4", "C6H8O7", "C2H4O2"
        ]
    bases = [
          "NaOH", "KOH",    "LiOH",   "Ba(OH)2", "Ca(OH)2", "Sr(OH)2",
          "NH3",  "CH3NH2", "Na2CO3", "NaHCO3",
        ]
    store=self.disect_form(form)
    if any(i in form for i in metal_elements) or 'NH4' in form:
      if any(i in form for i in super_sol_cat): return 'aq'
      elif 'CO3' in form: return 's'
      elif 'SO4' in form:
        if any(i in form for i in ['Ca','Pb','Ba']): return 's'
        else: return 'aq'
      elif any(i in form for i in ['Cl','Br','I']):
        if any(i in form for i in ['Pb','Ag']): return 's'
        else: return 'aq'
      elif form in acids or form in bases or 'CH3COO' in form: return 'aq'
    else:
      if form in confirmed: return confirmed[form]
      elif self.look_cate(form)=='noble gases': return 'g'
      elif form in acids or form in bases or 'CH3COO' in form or 'C2H3O2' in form: return 'aq'
    return '?'

  def disect_form(self,form):
    part=list(form)[::-1]; Mmu=1
    store={}; counter=0; local_mus=[];nex=1
    while counter<len(part):
      if part[counter]=='(':
        local_mus.pop()
      elif part[counter].isupper():
        name=part[counter]
        if name not in store: store[name]=0
        store[name]+=Mmu*math.prod(local_mus)*nex;nex=1
      elif part[counter+1].isupper() and part[counter].islower():
        name=part[counter+1]+part[counter]
        if name not in store: store[name]=0
        store[name]+=Mmu*math.prod(local_mus)*nex;nex=1
        counter+=1
      elif part[counter].isdigit():
        if part[counter+1]==')': local_mus.append(int(part[counter]))
        else: nex=int(part[counter])
      counter+=1
    return store

  def csal_to_form(self,csal):
    compounds=[]
    for i in csal:
      if i not in '><.*[]123-':
        if i.isupper():
          compounds.append(i)
        else: compounds[-1]+=i
    return self.list_to_form(compounds)
  def list_to_form(self,atoms):
    counts = Counter(atoms)
    if "C" in counts: order = ["C", "H"] + sorted(e for e in counts if e not in {"C","H"})
    else: order = sorted(counts)
    formula = ""
    for elem in order:
      if elem in counts:
        count = counts[elem]
        formula += elem + (str(count) if count > 1 else "")
    return formula


  def structure_identity(self,form):
    periodic_table = {
          "H":  ("nonmetal", 2.20),  "Li": ("metal", 0.98),   "Na": ("metal", 0.93),   "K":  ("metal", 0.82),
          "Be": ("metal", 1.57),     "Mg": ("metal", 1.31),   "Ca": ("metal", 1.00),   "B":  ("metalloid", 2.04),
          "C":  ("nonmetal", 2.55),  "N":  ("nonmetal", 3.04), "O": ("nonmetal", 3.44), "F":  ("nonmetal", 3.98),
          "Cl": ("nonmetal", 3.16),  "Br": ("nonmetal", 2.96), "I": ("nonmetal", 2.66), "S":  ("nonmetal", 2.58),
          "P":  ("nonmetal", 2.19),  "Fe": ("metal", 1.83),   "Cu": ("metal", 1.90),   "Zn": ("metal", 1.65),
          "Al": ("metal", 1.61),     "Si": ("metalloid", 1.90), "Mn": ("metal", 1.55),  "Co": ("metal", 1.88),
          "Ni": ("metal", 1.91),     "Cr": ("metal", 1.66),   "Ag": ("metal", 1.93),   "Au": ("metal", 2.54),
          "Hg": ("metal", 2.00),     "Pb": ("metal", 2.33),   "Sn": ("metal", 1.96),   "As": ("metalloid", 2.18),
          "Se": ("nonmetal", 2.55),  "Ti": ("metal", 1.54),   "V":  ("metal", 1.63),   "Ga": ("metal", 1.81),
          "Ge": ("metalloid", 2.01), "Sr": ("metal", 0.95),   "Ba": ("metal", 0.89),   "Rb": ("metal", 0.82),
          "Zr": ("metal", 1.33),     "Cd": ("metal", 1.69),   "Sb": ("metalloid", 2.05), "Te": ("metalloid", 2.10)
        }
    metal_elements = [
          "Li", "Na", "K", "Rb", "Cs", "Fr",
          "Be", "Mg", "Ca", "Sr", "Ba", "Ra",
          "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn",
          "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd",
          "La", "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg",
          "Ac", "Rf", "Db", "Sg", "Bh", "Hs", "Mt", "Ds", "Rg", "Cn",
          "Al", "Ga", "In", "Sn", "Tl", "Pb", "Bi", "Po", "Fl",
          "Ce", "Pr", "Nd", "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu",
          "Th", "Pa", "U", "Np", "Pu", "Am", "Cm", "Bk", "Cf", "Es", "Fm", "Md", "No", "Lr"
        ]
    n_atomic = {
        2:'Diatomic', 3:'Triatomic', 4:'Tetratomic', 5:'Pentatomic',6:'Hexatomic',7:'Heptatomic'
    }
    polyatomic_ions = [
        'NH4', 'SO4', 'ClO3', 'CO3', 'H2PO4', 'PO4', 'HPO4', 'SO3'
    ]
    store=self.disect_form(form)
    if any(i in metal_elements for i in store):
      if any(i not in metal_elements for i in store): return 'Ionic'
      elif len(list(store))==1: return 'Cation'
      elif all(i in metal_elements for i in store): return 'Metallic'
    elif all(i not in metal_elements for i in store):
      if len(list(store))==1:
        if self.look_cate(list(store)[0])=='noble gases': return 'Noble gas'
        elif store[list(store)[0]]>1: return n_atomic.get(store[list(store)[0]],'Polyatomic')+' COV'
        return 'Anion'
      elif any(ion in form for ion in polyatomic_ions): return 'Ionic'
      else:
        sumt=sum([store[i] for i in store])
        return n_atomic.get(sumt,'Polyatomic')+' COV'
    return 'Unknown'
  def round_sf(self,x, sf=5):
    if x == 0: return 0
    return round(x, sf - int(math.floor(math.log10(abs(x)))) - 1)

  def display_table(self):
    Mx_=max([max([len(j) for j in self.table[i]]) for i in self.table])+4
    print('-'*(Mx_+1)*len(list(self.table)))
    for i in self.table:
      if i=='empty':print(end=' '.center(Mx_)+'|')
      else: print(end=i.center(Mx_)+'|')
    print()
    print('-'*(Mx_+1)*len(list(self.table)))
    for i in range(len(self.table[list(self.table)[0]])):
      for name in self.table:
        if self.table[name][i]=='empty': print(end=' '.center(Mx_)+'|')
        else: print(end=self.table[name][i].center(Mx_)+'|')
      print()

  def local_values(self,equa):
    left,right=equa.split('>')
    self.left_basis=[];self.right_basis=[]
    tokens=self.tokenize(left);self.local_left=[]
    for token in tokens:
      self.local_left.append(self.peramitize(token))
      self.left_basis.append(self.peramitize(token))
    tokens=self.tokenize(right);self.local_right=[]
    for token in tokens:
      self.local_right.append(self.peramitize(token))
      self.right_basis.append(self.peramitize(token))
  def tokenize(self,equa):
    tokens=['']
    for i in equa:
      if i in '+>': tokens.append('')
      else: tokens[-1]+=i
    return tokens
  def reorg(self):
    reorg_left,reorg_right=self.data_ori.split('>')
    reorg_left=self.tokenize(reorg_left)
    for v,i in enumerate(self.local_left):
      for name in i:
        mu=self.local_left[v][name]//self.left_basis[v][name]
        if mu>1: reorg_left[v]=str(mu)+reorg_left[v]
        break
    reorg_left='+'.join(reorg_left)
    reorg_right=self.tokenize(reorg_right)
    for v,i in enumerate(self.local_right):
      for name in i:
        mu=self.local_right[v][name]//self.right_basis[v][name]
        if mu>1: reorg_right[v]=str(mu)+reorg_right[v]
        break
    reorg_right='+'.join(reorg_right)
    self.main_equa=reorg_left+'>'+reorg_right
  def peramitize(self,part):
    if part[0].isdigit(): Mmu=int(part[0])
    else: Mmu=1
    part=list(part)[::-1]
    store={}; counter=0; local_mus=[];nex=1
    while counter<len(part):
      if part[counter]=='(':
        local_mus.pop()
      elif part[counter].isupper():
        name=part[counter]
        if name not in store: store[name]=0
        store[name]+=Mmu*math.prod(local_mus)*nex;nex=1
      elif part[counter+1].isupper() and part[counter].islower():
        name=part[counter+1]+part[counter]
        if name not in store: store[name]=0
        store[name]+=Mmu*math.prod(local_mus)*nex;nex=1
        counter+=1
      elif part[counter].isdigit():
        if part[counter+1]==')': local_mus.append(int(part[counter]))
        else: nex=int(part[counter])
      counter+=1
    return store
  def collectervise_diff(self):
    repeated=0
    while True:
      C_left=dict(sum((Counter(i) for i in self.local_left), Counter()))
      C_right=dict(sum((Counter(i) for i in self.local_right), Counter()))
      if len(C_left)!=len(C_right): print('Unbalancable, Extra chemical')
      C_diff={}
      for i in list(C_left): C_diff[i]=C_left[i]-C_right[i]
      if all(False if C_diff[i] else True for i in C_diff): break
      if sum(C_diff[i] for i in C_diff)>0:
        popu_right=[0 for i in range(len(self.local_right))]
        for v,i in enumerate(self.local_right):
          for name in list(i):
            if C_diff[name]-i[name]==0:popu_right[v]+=1
        pos=max(enumerate(popu_right), key=lambda x: x[1])[0]
        for i in self.local_right[pos]:self.local_right[pos][i]+=self.right_basis[pos][i]
      elif sum(C_diff[i] for i in C_diff)<=0:
        popu_left=[0 for i in range(len(self.local_left))]
        for v,i in enumerate(self.local_left):
          for name in list(i):
            if C_diff[name]+i[name]==0:popu_left[v]+=1
        pos=max(enumerate(popu_left), key=lambda x: x[1])[0]
        for i in self.local_left[pos]:self.local_left[pos][i]+=self.left_basis[pos][i]
      repeated+=1
      if repeated>30: break
    if repeated>30: print('UNBALANCED')
    #Ba(OH)2
    #{Ba:1,O:2,H:2}
    #HCl+Ba(OH)2>BaCl2+H2O

  def quantum_atom(self,n, l, m, Z=1):
    h = 6.62607015e-34
    ℏ = h / (2 * math.pi)
    𝜖 = 8.85418782e-12
    e = 1.60217663e-19
    m_e = 9.1093837e-31
    a0 = 4 * math.pi * 𝜖 * ℏ**2 / (m_e * e**2)

    def R(r):
        rho = 2 * Z * r / (n * a0)
        N = np.sqrt((2 * Z / (n * a0))**3 *
                    math.factorial(n - l - 1) / (2 * n * math.factorial(n + l)))
        L = genlaguerre(n - l - 1, 2 * l + 1)
        return N * np.exp(-rho / 2) * (rho**l) * L(rho)

    def Y(θ, φ):
        N = np.sqrt((2 * l + 1) / (4 * np.pi) *
                    math.factorial(l - abs(m)) / math.factorial(l + abs(m)))
        P = lpmv(abs(m), l, np.cos(θ))
        if m < 0: P *= (-1)**abs(m)
        return N * P * np.exp(1j * m * φ)

    return R, Y
  def sample_electron(self,n, l, m, Z=1, n_points=100000):
    R, Y = self.quantum_atom(n, l, m, Z)
    points = []
    r_max = 3 * n**2 / Z * 5.29e-11

    r_sample = np.linspace(0.1 * r_max, r_max, 100)
    θ_sample = np.linspace(0, np.pi, 50)
    φ_sample = np.linspace(0, 2 * np.pi, 50)
    P_max = 0

    for r in r_sample[:20]:
        for θ in θ_sample[:20]:
            for φ in φ_sample[:20]:
                prob = (abs(R(r))**2) * (abs(Y(θ, φ))**2) * r**2 * np.sin(θ)
                P_max = max(P_max, prob)

    P_max *= 1.2

    while len(points) < n_points:
        r = np.random.uniform(0.01 * r_max, r_max)
        theta = np.random.uniform(0, np.pi)
        phi = np.random.uniform(0, 2 * np.pi)

        prob = (abs(R(r))**2) * (abs(Y(theta, phi))**2) * r**2 * np.sin(theta)
        u = np.random.uniform(0, P_max)

        if u < prob:
            x = r * np.sin(theta) * np.cos(phi)
            y = r * np.sin(theta) * np.sin(phi)
            z = r * np.cos(theta)
            points.append([x, y, z])

    return np.array(points)
  def display_elec_cloud(self,N=1,L=0,M=0,Z=1,points=10000):
    points = self.sample_electron(n=N, l=L, m=M, Z=Z, n_points=points)

    points = points * 1e9

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(points[:, 0], points[:, 1], points[:, 2], s=0.1, alpha=0.5, c=points[:, 2], cmap='magma')
    ax.set_xlabel('X (nm)')
    ax.set_ylabel('Y (nm)')
    ax.set_zlabel('Z (nm)')
    ax.set_title(f'Electron Cloud: n={N}, l={L}, m={M}')
    plt.tight_layout()
    plt.show()

  def quantum_box(self,Lx=1,Ly=1,Lz=1,nx=1,ny=1,nz=1):
    def X(x):
      return math.sqrt(2/Lx)*math.sin(x*nx*math.pi/Lx)
    def Y(y):
      return math.sqrt(2/Ly)*math.sin(y*ny*math.pi/Ly)
    def Z(z):
      return math.sqrt(2/Lz)*math.sin(z*nz*math.pi/Lz)
  def display_particle_box(self,Lx=1,Ly=1,Lz=1,nx=1,ny=1,nz=1):
    psi = quantum_box(Lx,Ly,Lz,nx,ny,nz)

    x = np.linspace(0, Lx, 40)
    y = np.linspace(0, Ly, 40)
    z = np.linspace(0, Lz, 40)

    X, Y, Z = np.meshgrid(x, y, z)
    values = np.vectorize(psi)(X, Y, Z)
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')

    mask = np.abs(values) > 0.2
    ax.scatter(X[mask], Y[mask], Z[mask],
           c=values[mask], cmap='magma', alpha=0.5, s=1)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    plt.show()

  def display_form_sheet(self):
    #----------------------------------------------
    print('----------------------------------------------')
    display(Math(r"\textbf{Mole \; Calculations}"))
    print('----------------------------------------------')

    # Basic relationships
    display(Math(r"\textup{Definition: } 1\;mol = 6.022\times10^{23}\;\textup{particles (Avogadro's number)}"))
    print()

    display(Math(r"\textup{Basic \; Formulas:}"))
    display(Math(r"n = \frac{m}{M}"))
    display(Math(r"n = \frac{N}{N_A}"))
    display(Math(r"n = \frac{V}{V_m}"))
    print()

    display(Math(r"\textup{Where:}"))
    display(Math(r"\begin{aligned}"
                 r"n &: \text{ number of moles (mol)} \\"
                 r"m &: \text{ mass (g)} \\"
                 r"M &: \text{ molar mass (g/mol)} \\"
                 r"N &: \text{ number of particles (atoms, molecules, etc.)} \\"
                 r"N_A &: \text{ Avogadro's number } (6.022\times10^{23}) \\"
                 r"V &: \text{ volume of gas at given conditions (L)} \\"
                 r"V_m &: \text{ molar volume (22.4 L at STP)}"
                 r"\end{aligned}"))
    print()

    # Gas relationships
    display(Math(r"\textup{For \; gases:}"))
    display(Math(r"PV = nRT"))
    display(Math(r"R = 8.314\; J·mol^{-1}·K^{-1}"))
    display(Math(r"\text{At STP: } V_m = 22.4\;L/mol"))
    print()

    # Concentration
    display(Math(r"\textup{Concentration:}"))
    display(Math(r"C = \frac{n}{V}"))
    display(Math(r"where \; C \; in \; mol·L^{-1}, \; V \; in \; L"))
    print()

    # Empirical and molecular formula
    display(Math(r"\textup{Empirical \; \& \; Molecular \; Formulas:}"))
    display(Math(r"\text{Empirical formula → simplest whole number ratio of atoms.}"))
    display(Math(r"Molecular \; formula = \left(\frac{M_{molecular}}{M_{empirical}}\right)\times(\text{Empirical formula})"))
    print()

    # Percentage composition
    display(Math(r"\textup{Percentage \; composition:}"))
    display(Math(r"\% \text{ element} = \frac{\text{mass of element in 1 mol compound}}{M_{compound}} \times 100"))
    print()

    #----------------------------------------------
    print('----------------------------------------------')
    display(Math(r"\textbf{Quantum \; Mechanics}"))
    print('----------------------------------------------')

    # Schrödinger Equation
    display(Math(r"\textup{Time-independent \; Schrödinger's \; equation:}"))
    display(Math(r"\hat{H}\psi = E\psi"))
    print()
    display(Math(r"\textup{Hamiltonian \; operator:}"))
    display(Math(r"\hat{H} = -\frac{\hbar^{2}}{2m}\nabla^{2} + V"))
    print()

    # Potential energy
    display(Math(r"\textup{Potential \; energy:}"))
    display(Math(r"""Box \; Case:\; \;
    V(x)=\left\{\begin{matrix}
    0, & L_{min}\leq x\leq L_{max}\\
    \infty, & x<L_{min}\;\text{or}\;x>L_{max}
    \end{matrix}\right."""))
    display(Math(r"Atom \; Case:\; \; V(r)=-\frac{e^{2}}{4\pi\epsilon_{0}r}"))
    print()

    # Normalization condition
    display(Math(r"\textup{Normalization:}"))
    display(Math(r"\int_{-\infty}^{\infty} |\psi|^{2} dx = 1"))
    print()

    # Particle in a box
    display(Math(r"\textup{Particle \; in \; a \; box:}"))
    display(Math(r"\psi_n(x) = \sqrt{\frac{2}{L}} \sin\!\left(\frac{n\pi x}{L}\right), \; n=1,2,3,\dots"))
    display(Math(r"E_n = \frac{n^{2}h^{2}}{8mL^{2}}"))
    print()

    # Hydrogen atom
    display(Math(r"\textup{Hydrogen \; atom:}"))
    display(Math(r"E_n = -\frac{13.6}{n^{2}} \; eV"))
    display(Math(r"\psi_{n,l,m}(r,\theta,\phi) = R_{n,l}(r)Y_{l}^{m}(\theta,\phi)"))
    display(Math(r"R_{n,l}(r)=\frac{1}{r}\sqrt{\left ( \frac{2Z}{na_0} \right )^{3}\frac{(n-l-1)!}{2n(n+1)!}}\left (  \frac{2Zr}{na_0} \right )^{l}e^{-\frac{Zr}{na_{0}}}L_{n-l-1}^{2l+1}\left (\frac{2Zr}{na_{0}}  \right )"))
    display(Math(r"Y^{m}_{l}(\theta,\phi)=(-1)^{m}\sqrt{\frac{2l+1}{4\pi}\frac{(l-m)!}{(l+m)!}}L^{m}_{l}(cos(\theta))e^{im\phi}"))
    print()

    # Quantum numbers
    display(Math(r"\textup{Quantum \; numbers:}"))
    display(Math(r"\begin{aligned}"
                 r"n &: \text{Principal (energy level)}\\"
                 r"l &: \text{Azimuthal (orbital shape)}\\"
                 r"m_l &: \text{Magnetic (orientation)}\\"
                 r"m_s &: \text{Spin (+}\tfrac{1}{2}\text{ or -}\tfrac{1}{2}\text{)}"
                 r"\end{aligned}"))
    print()

    # Momentum and angular momentum
    display(Math(r"\textup{Operators:}"))
    display(Math(r"\hat{p}_x = -i\hbar\frac{\partial}{\partial x}"))
    display(Math(r"\hat{L}^2 = \hbar^2 l(l+1), \;\;\; \hat{L}_z = \hbar m_l"))
    print()

    # Probability interpretation
    display(Math(r"\textup{Probability \; density:}"))
    display(Math(r"P(x) = |\psi(x)|^{2}"))
    print()

    print('----------------------------------------------')
    display(Math(r"End \; of \; Formula \; Sheet"))
    print('----------------------------------------------')

chem()

#HCl+Ba(OH)2>BaCl2+H2O
#(NH4)2SO4+NaOH>Na2SO4+H2O+NH3
#CH3COOH+NaOH>CH3COONa+H2O
#CH4+O2>CO2+H2O

#example drawings
#Acetic acid C*>[C.>[OH]*2[O]]

         📦PARTICLE IN POTENTIAL WELL📦         
----------------------------------------------
x by y by z -length & quantum number
i.e. (3,3,3)&(1,2,1)


KeyboardInterrupt: Interrupted by user

In [ ]:
class draw_struc:
  def __init__(self,CSAL):
    self.bonds={'·':[],'◦':[],'•':[]}
    self.points=[]
    self.draw_(CSAL)
    self.draw_out()

  def draw_(self,csal,ref=[0,0]):
    secs=self.disect(csal); bon_t={1:'·',2:'◦',3:'•'}
    try: self.points.append(ref+[secs.pop(0)])
    except: print('ERROR IN',csal)
    for i in secs:
      bonding, dir, fp=self.identify(i)
      dy,dx=dir
      self.bonds[bon_t.get(bonding,'·')].append([ref[0]+dy,ref[1]+dx])
      new_csal=i[fp:-1]
      self.draw_(new_csal,ref=[ref[0]+dy*2,ref[1]+dx*2])
    return

  def identify(self,csal):
    bonding=0;direction=[-1,0];final_p=0
    for v,i in enumerate(csal):
      if i=='*': direction[0]=-1
      elif i=='>': direction[1]=1
      elif i=='<': direction[1]=-1
      elif i=='.': direction[0]=1
      elif i=='-': direction[0]=0
      elif i.isdigit(): bonding=int(i)
      elif i=='[':
        final_p=v+1
        break
    return bonding, direction, final_p

  def disect(self,part):
    sections=['']; counter=0
    for i in part:
      if i in '*><.-' and not counter and sections[-1] not in '*><.-': sections.append(i)
      else:sections[-1]+=i
      if i==']': counter-=1
      elif i=='[': counter+=1
    return sections

  def draw_out(self):
    mx=min([i[1] for i in self.points])
    my=min([i[0] for i in self.points])
    Mx=max([i[1] for i in self.points])+abs(mx)+1
    My=max([i[0] for i in self.points])+abs(my)+1
    grid=[[' ' for __ in range(Mx)] for _ in range(My)]
    for i in self.points: grid[i[0]+abs(my)][i[1]+abs(mx)]=i[2]
    for i in self.bonds:
      for j in self.bonds[i]:
        grid[j[0]+abs(my)][j[1]+abs(mx)]=i
    for _ in grid:
      print(end=' ')
      for __ in _:
        print(end=__+' ')
      print()

#C.>[C>[C.>[C]]]
#O>[H]>[H]
#* means down

draw_struc('C*[C*>2[C*[Cl]]].>2[C*>[C*2[C*<[C]]]]')


     Cl     
     ·     
     C     
   ◦   ·   
 C       C 
 ·       ◦ 
 C       C 
   ◦   ·   
     C     
